In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt5

In [2]:
import sys
import time

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [31]:
def pulse( x ):
    y = np.zeros( x.shape )
    y[ np.where( (-1/2<x) & (x<1/2) ) ] = 1
    return y

def pulse2( x ):
    if (-1/2<x) and (x<1/2):
        y = 1
    else:
        y = 0
    return y

In [5]:
fmin = 5.725e9
fMAX = 5.875e9
fc = 5.8e9
B = fMAX - fmin
c = 3e8
kc = 4 * np.pi * fc / c
fs = 10e6

rail = 0
truck = 1
plane = 2
vehicule = rail

if vehicule == rail:
    T = 5e-3
    rampsPerFile = 200
    V = 0.1
if vehicule == truck:
    T = 5e-3
    rampsPerFile = 200
    V = 3
if vehicule == plane:
    T = 600e-6
    rampsPerFile = 1500
    V = 40

alpha = 2 * (fMAX - fmin) / T
samplesPerRamp = int( T * fs )
samplesPerFile = rampsPerFile * samplesPerRamp
samplesPerUpRamp = int( samplesPerRamp / 2 )

print("T = {}, samplesPerRamp = {:.2f}, samplesPerFile = {:.2f}".format(T, samplesPerRamp, samplesPerFile))

t1 = np.arange( 0, T, 1/fs)
t2 = np.arange( -T/2, T/2, 1/fs)

T = 0.005, samplesPerRamp = 50000.00, samplesPerFile = 10000000.00


In [6]:
def sb1( t, r ):
    tau = 2 * r / c
    y = np.exp( -1j * 2  * np.pi * fc * tau ) \
            * np.exp( 1j * np.pi * alpha * tau**2 ) \
            * np.exp(  -1j * 2 * np.pi * alpha * t * tau )
    return y

def sb2( t, r ):
    y = np.zeros( t.shape, dtype=complex )
    tau = 2 * r / c
    idx = np.where( t < tau )
    y[idx] = np.exp( -1j * 2  * np.pi * fc * (T-tau) ) \
            * np.exp( 1j * np.pi * alpha * (T-tau)**2 ) \
            * np.exp(  -1j * 2 * np.pi * alpha * t[idx] * (T-tau) )
    idx = np.where( t > tau )
    y[idx] = np.exp( -1j * 2  * np.pi * fc * tau ) \
            * np.exp( 1j * np.pi * alpha * tau**2 ) \
            * np.exp(  -1j * 2 * np.pi * alpha * (t[idx]-tau) * tau )
    return y

In [7]:
plt.figure()
r0 = 50.1
tau0 = 2 * r0 / c
print( "tau = {}, f = {}".format(tau0, alpha*tau0) )
plt.subplot( 211 )
plt.plot( np.real( sb1( t1[0:3000], r0 ) ), '.' )
plt.plot( np.real( sb2( t1[0:3000], r0 ) ) )
ax = plt.gca()
plt.subplot( 212, sharex=ax )
plt.plot( np.imag( sb1( t1[0:3000], r0 ) ) )
plt.plot( np.imag( sb2( t1[0:3000], r0 ) ) )
ax = plt.gca()
ax.set_xlim(0, 100)

tau = 3.34e-07, f = 20040.0


(0, 100)

In [8]:
plt.figure()
r0 = 50.1
tau0 = 2 * r0 / c
print( "tau = {}, f = {}".format(tau0, alpha*tau0) )
sb1_t1 = np.real( sb1( t1[0:3000], r0 ) )
sb2_t1 = np.real( sb2( t1[0:3000], r0 ) )
plt.subplot(311)
plt.plot( 20 * np.log10(np.abs(np.fft.ifft( sb1_t1 ) ) ) )
plt.plot( 20 * np.log10(np.abs(np.fft.ifft( sb2_t1 ) ) ) )
plt.subplot(312)
plt.plot( np.real(np.fft.ifft( sb1_t1 ) ) )
plt.plot( np.real(np.fft.ifft( sb2_t1 ) ) )
plt.subplot(313)
plt.plot( np.imag(np.fft.ifft( sb1_t1 ) ) )
plt.plot( np.imag(np.fft.ifft( sb2_t1 ) ) )

tau = 3.34e-07, f = 20040.0


In [9]:
def sr( t, r ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * (t[0:samplesPerUpRamp]) * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * (t[samplesPerUpRamp:2*samplesPerUpRamp]) * tau )
    
    return y

In [10]:
def srn( t, r, n ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * (t[0:samplesPerUpRamp] + n*T + tau) * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * (t[samplesPerUpRamp:2*samplesPerUpRamp] + n*T + tau) * tau )
    
    return y

# Build data

In [11]:
a = 0.603354564402
b = a * 180 / (30 * np.pi) # see the Newton method at the bottom of the notebook
def wa( az, rg0 ):
    Rn = ( rg0**2 + az**2 )**0.5
    wa = np.sinc( b * np.arccos( rg0 / Rn ) )**2
    return wa

In [27]:
plt.figure()
az = np.arange( -100, 100, 0.1 )
rg0 = 100
plt.plot( az, wa( az, rg0 ) )

In [12]:
nbFiles = 20

In [13]:
hPlane = 90
hCorner = 90
gr_corner = 100
rg_corner = ( (hPlane - hCorner)**2 + gr_corner**2 )**0.5
az_corner = V * T * nbFiles * rampsPerFile / 2
noise = 1
print("rg_corner = {:.2f}, gr_corner = {:.2f}, az_corner = {:.2f}".format(rg_corner, gr_corner, az_corner))

rg_corner = 100.00, gr_corner = 100.00, az_corner = 1.00


In [14]:
Laz = nbFiles * rampsPerFile * T * V
phi = np.arctan( (Laz/2) / rg_corner )
print( "Laz = {:.0f}, phi = {:.1f}°".format( Laz, phi * 180 / np.pi ) )

Laz = 2, phi = 0.6°


In [33]:
adc = np.zeros( (nbFiles * rampsPerFile, samplesPerRamp), dtype=complex )
for ramp in range( nbFiles * rampsPerFile):
    azn = ramp * T * V
    Rn = ( rg_corner**2 + (azn - az_corner)**2 )**0.5
    adc[ ramp,  0 : samplesPerRamp ] = sb1( t1, Rn ) * wa( azn - az_corner, rg_corner ) \
    + np.random.randn( samplesPerRamp ) * noise / (Rn**2)
    file = ramp/rampsPerFile
    if file == np.floor(file):
        phi = np.arccos( rg_corner / Rn ) * 180 / np.pi
        print( "{}, azn = {:.2f}, wa = {:.1f}, phi = {:.1f}, Rn = {:.1f}".format( 
            file, azn, wa( azn - az_corner, rg_corner ), phi, Rn ) )

0.0, azn = 0.00, wa = 1.0, phi = 0.6, Rn = 100.0
1.0, azn = 0.10, wa = 1.0, phi = 0.5, Rn = 100.0
2.0, azn = 0.20, wa = 1.0, phi = 0.5, Rn = 100.0
3.0, azn = 0.30, wa = 1.0, phi = 0.4, Rn = 100.0
4.0, azn = 0.40, wa = 1.0, phi = 0.3, Rn = 100.0
5.0, azn = 0.50, wa = 1.0, phi = 0.3, Rn = 100.0
6.0, azn = 0.60, wa = 1.0, phi = 0.2, Rn = 100.0
7.0, azn = 0.70, wa = 1.0, phi = 0.2, Rn = 100.0
8.0, azn = 0.80, wa = 1.0, phi = 0.1, Rn = 100.0
9.0, azn = 0.90, wa = 1.0, phi = 0.1, Rn = 100.0
10.0, azn = 1.00, wa = 1.0, phi = 0.0, Rn = 100.0
11.0, azn = 1.10, wa = 1.0, phi = 0.1, Rn = 100.0
12.0, azn = 1.20, wa = 1.0, phi = 0.1, Rn = 100.0
13.0, azn = 1.30, wa = 1.0, phi = 0.2, Rn = 100.0
14.0, azn = 1.40, wa = 1.0, phi = 0.2, Rn = 100.0
15.0, azn = 1.50, wa = 1.0, phi = 0.3, Rn = 100.0
16.0, azn = 1.60, wa = 1.0, phi = 0.3, Rn = 100.0
17.0, azn = 1.70, wa = 1.0, phi = 0.4, Rn = 100.0
18.0, azn = 1.80, wa = 1.0, phi = 0.5, Rn = 100.0
19.0, azn = 1.90, wa = 1.0, phi = 0.5, Rn = 100.0


# Backprojection

In [16]:
srf = np.fft.ifft( adc[:, 0:samplesPerUpRamp], axis=1 )
print( "srf.shape = {}".format(srf.shape))

srf.shape = (4000, 25000)


## Image geometry

In [45]:
x_min = -6
x_max = 8
r_min = 95
r_max = 105

#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max]
extent=[r_min,r_max,x_max,x_min]

d_x = 0.01
d_r = 0.1

x = np.arange( x_min, x_max, d_x )
x = x.reshape( x.size, 1 )
r = np.arange( r_min, r_max, d_r )
r = r.reshape( 1, r.size )

## Range focalization

In [46]:
overSamplingRatio = 4
Nf = srf.shape[1]
nbPointsResampled = overSamplingRatio * Nf
rangeResolution = c / (2 * B)
r_base = np.arange( 0, Nf ) * rangeResolution
r_over = np.arange( 0, nbPointsResampled ) * rangeResolution / overSamplingRatio
print( "range from {:.2f}m to {:.2f}m, resolution = {}m, oversampled = {}m, ".format(
    r_over[0], r_over[-1], rangeResolution, rangeResolution / overSamplingRatio ) )

range from 0.00m to 24999.75m, resolution = 1.0m, oversampled = 0.25m, 


In [47]:
Naz = nbFiles * rampsPerFile
xa_vec = np.arange( Naz ) * T * V
nbLoops = xa_vec.size
loop = 0
img  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}, x.shape = {}, r.shape = {}".format( img.shape, x.shape, r.shape ) )
print( "x from {} to {}, r from {} to {}".format(x[0,0], x[-1,0], r[0,0], r[0,-1]) )

phi = 30 * np.pi / 180
r0 = rg_corner
Wx_r0 = r0 * np.tan(phi) * 2
Wx_r0 = 10
print("Wx_r0 = {:.2f}".format(Wx_r0))

t = time.time()

for xa in xa_vec:
    if loop%1000 == 0:
        print( "{} / {}".format(loop, nbLoops ) )
    img += np.exp( 1j * kc * (r**2 + (xa-x)**2 )**0.5 ) \
    * np.interp( (r**2 + (xa-x)**2 )**0.5, r_over, signal.resample( srf[loop,:], nbPointsResampled  ) ) \
    * pulse( (xa-x) / Wx_r0 )
    #img += np.exp( 1j * kc * (r**2 + (xa-x)**2 )**0.5 ) \
    #* np.interp( (r**2 + (xa-x)**2 )**0.5, r_base, srf[loop,:] )
    loop += 1
# np.exp( -1j * kc * r)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

img.shape = (1400, 100), x.shape = (1400, 1), r.shape = (1, 100)
x from -6.0 to 7.989999999999702, r from 95.0 to 104.89999999999944
Wx_r0 = 10.00
0 / 4000
1000 / 4000
2000 / 4000
3000 / 4000
execution time = 53.66207575798035


In [48]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( img ) ), cmap='jet', extent=extent )
plt.grid()
plt.colorbar(orientation="horizontal")

/opt/Py3Venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  


# Newton's method
f(x) = sin(x) / x

f'(x) = cos(x) / x - sin(x) / x^2

x(n+1) = x(n) - f(x(n)) / f'(x(n))

x(n+1) = x(n) - [f(x(n)) - y0] / f'(x(n))

In [ ]:
y0 = 0.1
def newton( xn ):
    f_xn = np.sinc( xn )
    #f_prime_xn = np.cos(xn) / xn - np.sin(xn) / xn**2
    f_prime_xn = np.cos(np.pi * xn) / xn - np.sin(np.pi*xn) / (np.pi*xn**2)
    xn_plus_1 = xn - ( f_xn - y0 ) / f_prime_xn
    return xn_plus_1

In [ ]:
x = 1
for k in range( 10 ):
    x = newton( x )
    print( x )

In [ ]:
a = 0.603354564402 # y0 = 0.5
a = 0.90792862378 # y0 = 0.1
b = a * 180 / (30 * np.pi)

In [ ]:
x = 30 * np.pi / 180
np.sinc( b * x )